# Graden Groups

In [1]:
import numpy as np

## Part 1

In [2]:
def parse_input(n_sample=None, file_path="Garden_Groups.txt", mode="r"):
	if n_sample is not None:
		file_path = f"sample{n_sample}.txt"
		with open(file_path, mode) as f:
			return f.read().splitlines()

	else:
		with open(file_path, mode) as f:
			return f.read().splitlines()

In [3]:
def get_group(garden, x, y, group_type=None, group_perimeter=None, group_edge_positions=None, visited=None):
	
	directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
	
	if visited is None:
		visited = set()
	if group_type is None:
		group_type = garden[x][y]
	if group_perimeter is None:
		group_perimeter = 0
	if group_edge_positions is None:
		group_edge_positions = set()

	if (x, y) in visited:
		return visited
	
	visited.add((x, y))
	group_edge_positions.add((x, y))

	for (dx, dy) in directions:
		nx, ny = x + dx, y + dy
		if nx < 0 or nx >= len(garden) or ny < 0 or ny >= len(garden[0]):
			group_perimeter += 1
			group_edge_positions.add((x, y))
			continue
		elif (nx, ny) in visited:
			continue
		elif garden[nx][ny] != group_type:
			group_perimeter += 1
			group_edge_positions.add((x, y))
			continue
		else:
			_, group_perimeter, group_edge_positions = get_group(garden, nx, ny, group_type, group_perimeter, group_edge_positions, visited)
		
	return visited, group_perimeter, group_edge_positions

In [4]:
def get_all_groups(garden):
	garden_dims = (len(garden), len(garden[0]))

	all_groups = []
	all_edges = []
	all_edge_positions = []

	grouped_elements = [[False for _ in range(garden_dims[1])] for _ in range(garden_dims[0])]

	for x in range(garden_dims[0]):
		for y in range(garden_dims[1]):
			if grouped_elements[x][y]:
				continue
			group, edges, edge_pos = get_group(garden, x, y)
			all_groups.append((garden[x][y], group))
			all_edges.append(edges)
			all_edge_positions.append(edge_pos)

			for (i, j) in group:
				grouped_elements[i][j] = True

	return all_groups, all_edges, all_edge_positions

In [5]:
def get_cost(groups, edges, verbose=False):
	cost = 0 
	for (group_type, group), edge in zip(groups, edges):
		if verbose:
			print(group_type, len(group), edge, len(group) * edge)
		cost += len(group) * edge

	return cost

In [6]:
def part1(sample=None, verbose=False):
	garden = parse_input(sample)
	groups, edges, edge_pos = get_all_groups(garden)
	cost = get_cost(groups, edges, verbose=verbose)
	return cost

In [7]:
part1()

1402544

## Part 2

In [60]:
def get_edge_types(groups):
	edge_types = dict()
	directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
	directions_dict = {
		(0, 1): "R",
		(0, -1): "L",
		(1, 0): "D", 
		(-1, 0): "U"
	}

	for group_type, group_positions in groups:

		for direction in directions:
			dx, dy = direction
			edge_type = directions_dict[direction]
			edge_positions = []
			for pos in group_positions:
				x, y = pos
				nx, ny = x + dx, y + dy
				if (nx, ny) not in group_positions:
					edge_positions.append((x, y))
			
			if group_type not in edge_types:
				edge_types[group_type] = [[edge_type, edge_positions]]

			else:
				edge_types[group_type].append([edge_type, edge_positions])

	return edge_types

In [61]:
sample = 1
garden = parse_input(sample)
groups, edges, edge_pos = get_all_groups(garden)

In [62]:
edge_types = get_edge_types(groups)

In [67]:
groups

[('A', {(0, 0), (0, 1), (0, 2), (0, 3)}),
 ('B', {(1, 0), (1, 1), (2, 0), (2, 1)}),
 ('C', {(1, 2), (2, 2), (2, 3), (3, 3)}),
 ('D', {(1, 3)}),
 ('E', {(3, 0), (3, 1), (3, 2)})]

In [82]:
import numpy as np

# Example arrays
array1 = np.array([1, 2, 3, 4])
array2 = np.array([1, 2, 3, 4])

array2 = array2 - 1

output1 = np.setdiff1d(array1, array2)
output2 = np.setdiff1d(array2, array1)

print(output1, output2)

[4] [0]


In [133]:
costs = []
all_edges = []

for plant_type in edge_types.keys():
	plant_surface = [len(group) for group_type, group in groups if group_type == plant_type][0]
	plant_edges = 0
	

	for edge_side, edge_positions in edge_types[plant_type]:
	
		if edge_side in ["U", "D"]:
			positions = sorted(edge_positions, key=lambda x: x[1])

		elif edge_side in ["L", "R"]:
			positions = sorted(edge_positions, key=lambda x: x[0])

		if len(positions) == 1:
			plant_edges += 1

		for i in range(len(positions) - 1):
			pos1 = positions[i]
			pos2 = positions[i + 1]

			if edge_side in ["U", "D"]:
				if pos1[1] - pos2[1] != 1:
					plant_edges += 1
			
			elif edge_side in ["L", "R"]:
				if pos1[0] - pos2[0] != 1:
					plant_edges += 1

	all_edges.append(plant_edges)
	costs.append([plant_surface * plant_edges])

In [134]:
all_edges

[8, 4, 6, 4, 6]

In [128]:
plant_surface

3